In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv('processed.cleveland.data', header=None)

In [7]:
df.columns = ['age', 'sex', 'cp', 'restbp', 'chol', 'fbs',
              'restecg', 'thalach', 'exang', 'oldpeak',
              'slope', 'ca', 'thal', 'hd']

In [8]:
df = df.replace('?', np.nan)

In [9]:
df = df.apply(pd.to_numeric, errors='ignore')

In [10]:
df = df.dropna()

In [11]:
df['hd'] = df['hd'].apply(lambda x: 1 if x != 0 else 0)

In [12]:
df = pd.get_dummies(df, columns=['cp', 'restecg', 'slope', 'thal'], drop_first=True)

In [15]:
df.shape

## Implementación del programa

In [16]:
np.random.seed(123)

In [17]:
# Creación de la variable T
df['T'] = np.random.choice([0, 1], size=len(df))

In [19]:
# Definimos el error aleatorio:
eps = np.random.normal(loc=0, scale=1, size=len(df))

In [20]:
# Creación de la variable Y:
df['Y'] = (
    df['T'] * (1 + 0.05*df['age'] + 0.3*df['sex'] + 0.2*df['restbp'])
    + 0.5*df['oldpeak']
    + eps
)

In [23]:
# OLS
Y = df['Y']
X = df['T']
X = sm.add_constant(X)

In [25]:
modelo_ols = sm.OLS(Y, X).fit()
print(modelo_ols.summary())

In [27]:
# Random Forest
X = df.drop(columns=['Y', 'T'])

In [28]:
rf_treated = RandomForestRegressor(n_estimators=100, random_state=123)
rf_control = RandomForestRegressor(n_estimators=100, random_state=123)

In [30]:
rf_treated.fit(X[df['T'] == 1], df.loc[df['T'] == 1, 'Y'])
rf_control.fit(X[df['T'] == 0], df.loc[df['T'] == 0, 'Y'])

In [31]:
y_pred_treated = rf_treated.predict(X)
y_pred_control = rf_control.predict(X)

In [32]:
df['ITE'] = y_pred_treated - y_pred_control

In [33]:
ATE = df['ITE'].mean()

In [37]:
plt.hist(df['ITE'], bins=30, edgecolor='black')
plt.title('Distribución de efectos individuales (ITE)')
plt.xlabel('Efecto del tratamiento estimado')
plt.ylabel('Frecuencia')
plt.tight_layout()
plt.savefig('histograma_efectos_individuales_python.png', dpi=300)
plt.show()

In [39]:
# Representative tree
tree_model = DecisionTreeRegressor(max_depth=2, random_state=123)
tree_model.fit(X, df['ITE'])

plt.figure(figsize=(14, 6))
plot_tree(tree_model,
          feature_names=X.columns,
          filled=True,
          rounded=True,
          fontsize=10)
plt.title('Árbol representativo')
plt.savefig('arbol_representativo_python.png', dpi=300)
plt.show()

In [43]:
# La importancia
importances = tree_model.feature_importances_

In [44]:
feat_importances = pd.DataFrame({
    'Variable': X.columns,
    'Importancia': importances
}).sort_values(by='Importancia', ascending=False)

In [45]:
plt.figure(figsize=(10,5))
plt.bar(feat_importances['Variable'], feat_importances['Importancia'], color='skyblue', edgecolor='black')
plt.xticks(rotation=45, ha='right')
plt.title('Importancia de las variables en el árbol (max_depth=2)')
plt.ylabel('Importancia relativa')
plt.tight_layout()
plt.savefig('importancia_variables_arbol.png', dpi=300)
plt.show()

In [47]:
# distribución de covariables
scaler = StandardScaler()
X_std = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [48]:
df['tercil'] = pd.qcut(df['ITE'], q=3, labels=['bajo', 'medio', 'alto'])

In [50]:
means_by_tercile = X_std.copy()
means_by_tercile['tercil'] = df['tercil']
mean_table = means_by_tercile.groupby('tercil', observed=True).mean()

In [51]:
plt.figure(figsize=(12, 5))
sns.heatmap(mean_table, cmap='coolwarm', annot=True, fmt='.2f', cbar_kws={'label': 'Valor promedio'})
plt.title('Distribución de covariables estandarizadas por terciles del efecto del tratamiento')
plt.xlabel('Covariables')
plt.ylabel('Terciles del efecto del tratamiento')
plt.tight_layout()
plt.savefig('heatmap_python.png', dpi=300)
plt.show()

### Covariables

El mapa de calor muestra cómo varían las covariables estandarizadas según los niveles del efecto del tratamiento. Las tonalidades rojas representan valores promedio más altos y las azules valores más bajos. Se observa que la variable **restbp** tiende a tener valores más elevados en el grupo con efecto alto, lo que refuerza la evidencia de que las personas con mayor presión arterial en reposo presentan una respuesta más fuerte al tratamiento.